# Chapter 8: Data Fetching Fundamentals

Data fetching is the backbone of modern web applications. Whether you're displaying user profiles, product catalogs, blog posts, or analytics dashboards, you need to fetch data from various sources efficiently. Next.js provides powerful data fetching capabilities that work seamlessly with Server and Client Components.

By the end of this chapter, you'll master data fetching in Next.js, understand caching and revalidation strategies, and know how to build fast, responsive applications with excellent user experience.

## 8.1 Fetching Data in Server Components

Server Components can fetch data directly on the server, which is one of their biggest advantages. This approach eliminates the need for client-side loading states and provides excellent performance and SEO.

### Basic Server-Side Data Fetching

Server Components use standard async/await patterns for data fetching:

```tsx
// app/users/page.tsx

interface User {
  id: string;
  name: string;
  email: string;
  role: string;
  createdAt: string;
}

// Simulated data fetch (in real app, this would be from a database or API)
async function fetchUsers(): Promise<User[]> {
  // Simulate API delay
  await new Promise(resolve => setTimeout(resolve, 1000));
  
  return [
    { id: '1', name: 'Alice Johnson', email: 'alice@example.com', role: 'Admin', createdAt: '2024-01-15' },
    { id: '2', name: 'Bob Smith', email: 'bob@example.com', role: 'User', createdAt: '2024-01-16' },
    { id: '3', name: 'Charlie Brown', email: 'charlie@example.com', role: 'User', createdAt: '2024-01-17' },
    { id: '4', name: 'Diana Prince', email: 'diana@example.com', role: 'Editor', createdAt: '2024-01-18' },
  ];
}

export default async function UsersPage() {
  // Data is fetched on the server
  const users = await fetchUsers();

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="mb-8">
        <h1 className="text-3xl font-bold text-gray-900 mb-2">
          Users Management
        </h1>
        <p className="text-gray-600">
          Manage and view all users in your organization
        </p>
      </div>

      {/* Users table */}
      <div className="bg-white rounded-lg shadow-md overflow-hidden">
        <table className="w-full">
          <thead className="bg-gray-50">
            <tr>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                Name
              </th>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                Email
              </th>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                Role
              </th>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">
                Created
              </th>
            </tr>
          </thead>
          <tbody className="divide-y divide-gray-200">
            {users.map((user) => (
              <tr key={user.id} className="hover:bg-gray-50">
                <td className="px-6 py-4 whitespace-nowrap">
                  <div className="text-sm font-medium text-gray-900">{user.name}</div>
                </td>
                <td className="px-6 py-4 whitespace-nowrap">
                  <div className="text-sm text-gray-500">{user.email}</div>
                </td>
                <td className="px-6 py-4 whitespace-nowrap">
                  <span className="px-2 inline-flex text-xs leading-5 font-semibold rounded-full bg-blue-100 text-blue-800">
                    {user.role}
                  </span>
                </td>
                <td className="px-6 py-4 whitespace-nowrap text-sm text-gray-500">
                  {new Date(user.createdAt).toLocaleDateString()}
                </td>
              </tr>
            ))}
          </tbody>
        </table>
      </div>
    </div>
  );
}
```

### Fetching from External APIs

Server Components can fetch data from external APIs without CORS issues:

```tsx
// app/posts/page.tsx

interface Post {
  userId: number;
  id: number;
  title: string;
  body: string;
}

// Fetch data from external API
async function fetchPosts(): Promise<Post[]> {
  const response = await fetch('https://jsonplaceholder.typicode.com/posts', {
    cache: 'force-cache', // Cache the response
  });

  if (!response.ok) {
    throw new Error(`HTTP error! status: ${response.status}`);
  }

  return response.json();
}

export default async function PostsPage() {
  const posts = await fetchPosts();

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Blog Posts</h1>

      <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6">
        {posts.map((post) => (
          <article 
            key={post.id}
            className="bg-white rounded-lg shadow-md p-6 hover:shadow-lg transition-shadow"
          >
            <h2 className="text-xl font-semibold text-gray-900 mb-3 line-clamp-2">
              {post.title}
            </h2>
            <p className="text-gray-600 line-clamp-4 mb-4">
              {post.body}
            </p>
            <a 
              href={`/posts/${post.id}`}
              className="text-blue-600 hover:text-blue-800 font-medium"
            >
              Read more →
            </a>
          </article>
        ))}
      </div>
    </div>
  );
}
```

### Direct Database Access

Server Components can access databases directly, providing better performance and security:

```tsx
// app/dashboard/analytics/page.tsx

import { db } from '@/lib/db';
import { revenue, users, orders } from '@/lib/schema';

// Fetch analytics data directly from database
async function fetchAnalytics() {
  const [totalRevenue, totalUsers, totalOrders] = await Promise.all([
    // Total revenue
    db.select({ value: revenue.amount })
      .from(revenue)
      .then(results => results.reduce((sum, r) => sum + Number(r.value), 0)),
    
    // Total users
    db.select({ count: users.id })
      .from(users)
      .then(results => results.length),
    
    // Total orders
    db.select({ count: orders.id })
      .from(orders)
      .then(results => results.length),
  ]);

  return {
    totalRevenue,
    totalUsers,
    totalOrders,
  };
}

export default async function AnalyticsPage() {
  const analytics = await fetchAnalytics();

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">
        Analytics Dashboard
      </h1>

      <div className="grid grid-cols-1 md:grid-cols-3 gap-6">
        <AnalyticsCard
          title="Total Revenue"
          value={`$${analytics.totalRevenue.toLocaleString()}`}
          change="+12.5%"
          positive
        />
        <AnalyticsCard
          title="Total Users"
          value={analytics.totalUsers.toLocaleString()}
          change="+8.2%"
          positive
        />
        <AnalyticsCard
          title="Total Orders"
          value={analytics.totalOrders.toLocaleString()}
          change="-3.1%"
          positive={false}
        />
      </div>
    </div>
  );
}

function AnalyticsCard({ 
  title, 
  value, 
  change, 
  positive 
}: { 
  title: string; 
  value: string; 
  change: string; 
  positive: boolean;
}) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h3 className="text-sm font-medium text-gray-500 mb-2">{title}</h3>
      <p className="text-3xl font-bold text-gray-900 mb-2">{value}</p>
      <p className={`text-sm ${positive ? 'text-green-600' : 'text-red-600'}`}>
        {change} from last month
      </p>
    </div>
  );
}
```

### Error Handling in Server Components

Handle errors gracefully using try-catch blocks:

```tsx
// app/products/[id]/page.tsx

import { notFound } from 'next/navigation';

interface Product {
  id: string;
  name: string;
  description: string;
  price: number;
  imageUrl: string;
  stock: number;
}

async function fetchProduct(id: string): Promise<Product | null> {
  try {
    const response = await fetch(`https://api.example.com/products/${id}`, {
      next: { revalidate: 3600 }, // Revalidate every hour
    });

    if (response.status === 404) {
      return null;
    }

    if (!response.ok) {
      throw new Error(`Failed to fetch product: ${response.status}`);
    }

    return response.json();
  } catch (error) {
    console.error('Error fetching product:', error);
    throw error;
  }
}

export default async function ProductPage({ params }: { params: { id: string } }) {
  try {
    const product = await fetchProduct(params.id);

    if (!product) {
      notFound();
    }

    return (
      <div className="container mx-auto px-4 py-8">
        <div className="grid grid-cols-1 lg:grid-cols-2 gap-8">
          <div className="aspect-video bg-gray-100 rounded-lg overflow-hidden">
            <img
              src={product.imageUrl}
              alt={product.name}
              className="w-full h-full object-cover"
            />
          </div>

          <div>
            <h1 className="text-3xl font-bold text-gray-900 mb-4">
              {product.name}
            </h1>
            <p className="text-gray-600 mb-6">{product.description}</p>
            
            <div className="flex items-center justify-between mb-6">
              <span className="text-3xl font-bold text-blue-600">
                ${product.price.toFixed(2)}
              </span>
              <span className="text-sm text-gray-500">
                {product.stock} in stock
              </span>
            </div>

            <button className="w-full bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700 transition-colors">
              Add to Cart
            </button>
          </div>
        </div>
      </div>
    );
  } catch (error) {
    return (
      <div className="container mx-auto px-4 py-8">
        <div className="bg-red-50 border border-red-200 rounded-lg p-6">
          <h2 className="text-xl font-semibold text-red-900 mb-2">
            Error Loading Product
          </h2>
          <p className="text-red-700">
            We encountered an error while loading this product. Please try again later.
          </p>
        </div>
      </div>
    );
  }
}
```

## 8.2 Fetching Data in Client Components

Client Components fetch data on the client side using React hooks like `useEffect` and `useState`. This approach is useful for real-time data, user-specific content, or interactive features.

### Basic Client-Side Data Fetching

```tsx
// components/ClientPosts.tsx
"use client";

import { useState, useEffect } from 'react';

interface Post {
  id: number;
  title: string;
  body: string;
}

export function ClientPosts() {
  const [posts, setPosts] = useState<Post[]>([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState<string | null>(null);

  useEffect(() => {
    async function loadPosts() {
      try {
        setLoading(true);
        setError(null);

        const response = await fetch('https://jsonplaceholder.typicode.com/posts');

        if (!response.ok) {
          throw new Error(`HTTP error! status: ${response.status}`);
        }

        const data = await response.json();
        setPosts(data.slice(0, 10)); // Limit to 10 posts
      } catch (err) {
        setError(err instanceof Error ? err.message : 'An error occurred');
      } finally {
        setLoading(false);
      }
    }

    loadPosts();
  }, []);

  if (loading) {
    return (
      <div className="flex items-center justify-center py-12">
        <div className="animate-spin rounded-full h-12 w-12 border-b-2 border-blue-600"></div>
      </div>
    );
  }

  if (error) {
    return (
      <div className="bg-red-50 border border-red-200 rounded-lg p-6">
        <p className="text-red-700">Error: {error}</p>
      </div>
    );
  }

  return (
    <div className="space-y-4">
      {posts.map((post) => (
        <div key={post.id} className="bg-white rounded-lg shadow-md p-6">
          <h3 className="text-xl font-semibold text-gray-900 mb-2">{post.title}</h3>
          <p className="text-gray-600">{post.body}</p>
        </div>
      ))}
    </div>
  );
}
```

### Data Fetching with Custom Hooks

Create reusable custom hooks for data fetching:

```tsx
// hooks/useFetch.ts

import { useState, useEffect, useCallback } from 'react';

interface UseFetchResult<T> {
  data: T | null;
  loading: boolean;
  error: string | null;
  refetch: () => Promise<void>;
}

export function useFetch<T>(
  url: string,
  options?: RequestInit
): UseFetchResult<T> {
  const [data, setData] = useState<T | null>(null);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState<string | null>(null);

  const fetchData = useCallback(async () => {
    try {
      setLoading(true);
      setError(null);

      const response = await fetch(url, options);

      if (!response.ok) {
        throw new Error(`HTTP error! status: ${response.status}`);
      }

      const result = await response.json();
      setData(result);
    } catch (err) {
      setError(err instanceof Error ? err.message : 'An error occurred');
    } finally {
      setLoading(false);
    }
  }, [url, options]);

  useEffect(() => {
    fetchData();
  }, [fetchData]);

  return { data, loading, error, refetch: fetchData };
}
```

**Using the custom hook:**

```tsx
// app/users/page.tsx
"use client";

import { useFetch } from '@/hooks/useFetch';

interface User {
  id: string;
  name: string;
  email: string;
}

export default function UsersPage() {
  const { data: users, loading, error, refetch } = useFetch<User[]>(
    'https://jsonplaceholder.typicode.com/users'
  );

  if (loading) {
    return <div className="text-center py-12">Loading users...</div>;
  }

  if (error) {
    return (
      <div className="bg-red-50 border border-red-200 rounded-lg p-6">
        <p className="text-red-700 mb-4">Error: {error}</p>
        <button
          onClick={() => refetch()}
          className="px-4 py-2 bg-red-600 text-white rounded hover:bg-red-700"
        >
          Retry
        </button>
      </div>
    );
  }

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Users</h1>

      <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6">
        {users?.map((user) => (
          <div key={user.id} className="bg-white rounded-lg shadow-md p-6">
            <h3 className="text-xl font-semibold text-gray-900 mb-2">
              {user.name}
            </h3>
            <p className="text-gray-600">{user.email}</p>
          </div>
        ))}
      </div>
    </div>
  );
}
```

### Real-Time Data Fetching

Implement real-time data updates with polling or WebSocket:

```tsx
// components/StockTicker.tsx
"use client";

import { useState, useEffect } from 'react';

interface Stock {
  symbol: string;
  price: number;
  change: number;
}

export function StockTicker() {
  const [stocks, setStocks] = useState<Stock[]>([]);
  const [lastUpdate, setLastUpdate] = useState<Date>(new Date());

  useEffect(() => {
    // Fetch initial data
    fetchStocks();

    // Set up polling for real-time updates (every 5 seconds)
    const interval = setInterval(() => {
      fetchStocks();
    }, 5000);

    return () => clearInterval(interval);
  }, []);

  const fetchStocks = async () => {
    try {
      // Simulate real-time data fetch
      const response = await fetch('/api/stocks');
      const data = await response.json();
      setStocks(data);
      setLastUpdate(new Date());
    } catch (error) {
      console.error('Error fetching stocks:', error);
    }
  };

  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <div className="flex justify-between items-center mb-6">
        <h2 className="text-xl font-bold text-gray-900">Stock Prices</h2>
        <span className="text-sm text-gray-500">
          Last updated: {lastUpdate.toLocaleTimeString()}
        </span>
      </div>

      <div className="space-y-3">
        {stocks.map((stock) => (
          <div 
            key={stock.symbol}
            className="flex justify-between items-center p-3 bg-gray-50 rounded"
          >
            <div className="flex items-center space-x-3">
              <span className="font-semibold text-gray-900">{stock.symbol}</span>
            </div>
            <div className="flex items-center space-x-4">
              <span className="font-bold text-gray-900">
                ${stock.price.toFixed(2)}
              </span>
              <span className={stock.change >= 0 ? 'text-green-600' : 'text-red-600'}>
                {stock.change >= 0 ? '+' : ''}{stock.change.toFixed(2)}%
              </span>
            </div>
          </div>
        ))}
      </div>
    </div>
  );
}
```

### Infinite Scroll Data Fetching

Implement infinite scroll for large datasets:

```tsx
// components/InfiniteList.tsx
"use client";

import { useState, useEffect, useRef, useCallback } from 'react';

interface Item {
  id: number;
  name: string;
  description: string;
}

export function InfiniteList() {
  const [items, setItems] = useState<Item[]>([]);
  const [loading, setLoading] = useState(false);
  const [page, setPage] = useState(1);
  const [hasMore, setHasMore] = useState(true);
  const observer = useRef<IntersectionObserver | null>(null);

  const loadMore = useCallback(async () => {
    if (loading || !hasMore) return;

    setLoading(true);
    try {
      const response = await fetch(`/api/items?page=${page}&limit=10`);
      const data = await response.json();

      if (data.length === 0) {
        setHasMore(false);
      } else {
        setItems(prev => [...prev, ...data]);
        setPage(prev => prev + 1);
      }
    } catch (error) {
      console.error('Error loading items:', error);
    } finally {
      setLoading(false);
    }
  }, [loading, page, hasMore]);

  useEffect(() => {
    loadMore();
  }, []); // Load initial data

  // Set up intersection observer for infinite scroll
  const lastItemRef = useCallback(
    (node: HTMLDivElement | null) => {
      if (loading) return;

      if (observer.current) {
        observer.current.disconnect();
      }

      observer.current = new IntersectionObserver(
        (entries) => {
          if (entries[0].isIntersecting && hasMore) {
            loadMore();
          }
        },
        { threshold: 1.0 }
      );

      if (node) {
        observer.current.observe(node);
      }
    },
    [loading, hasMore, loadMore]
  );

  return (
    <div className="space-y-4">
      {items.map((item, index) => (
        <div
          key={item.id}
          ref={index === items.length - 1 ? lastItemRef : null}
          className="bg-white rounded-lg shadow-md p-6 hover:shadow-lg transition-shadow"
        >
          <h3 className="text-lg font-semibold text-gray-900 mb-2">
            {item.name}
          </h3>
          <p className="text-gray-600">{item.description}</p>
        </div>
      ))}

      {loading && (
        <div className="text-center py-4">
          <div className="inline-block animate-spin rounded-full h-8 w-8 border-b-2 border-blue-600"></div>
        </div>
      )}

      {!hasMore && items.length > 0 && (
        <div className="text-center text-gray-500 py-4">
          No more items to load
        </div>
      )}
    </div>
  );
}
```

## 8.3 Understanding fetch() and Built-in Caching

Next.js extends the native `fetch()` API with powerful caching and revalidation capabilities. Understanding these features is crucial for building performant applications.

### Caching Strategies

Next.js provides several caching strategies through the `fetch()` options:

**1. Force Cache (Default)**

```tsx
// Data is cached and reused until explicitly revalidated
async function getProducts() {
  const response = await fetch('https://api.example.com/products', {
    cache: 'force-cache', // Default behavior
  });
  return response.json();
}
```

**2. No Store**

```tsx
// Data is never cached, always fetched fresh
async function getRealTimeData() {
  const response = await fetch('https://api.example.com/realtime', {
    cache: 'no-store',
  });
  return response.json();
}
```

**3. Revalidate**

```tsx
// Data is cached for specified time, then revalidated
async function getProducts() {
  const response = await fetch('https://api.example.com/products', {
    next: { revalidate: 3600 }, // Revalidate every hour
  });
  return response.json();
}
```

### Time-Based Revalidation

Set up automatic revalidation at specific intervals:

```tsx
// app/dashboard/page.tsx

interface DashboardData {
  revenue: number;
  users: number;
  orders: number;
}

async function getDashboardData(): Promise<DashboardData> {
  // Data is cached for 5 minutes, then revalidated
  const response = await fetch('https://api.example.com/dashboard', {
    next: { revalidate: 300 }, // 5 minutes
  });

  if (!response.ok) {
    throw new Error('Failed to fetch dashboard data');
  }

  return response.json();
}

export default async function DashboardPage() {
  const data = await getDashboardData();

  const updateTime = new Date().toLocaleTimeString();

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="mb-6 flex justify-between items-center">
        <h1 className="text-3xl font-bold text-gray-900">Dashboard</h1>
        <p className="text-sm text-gray-500">
          Last updated: {updateTime}
        </p>
      </div>

      <div className="grid grid-cols-1 md:grid-cols-3 gap-6">
        <MetricCard 
          title="Revenue" 
          value={`$${data.revenue.toLocaleString()}`} 
          description="Total revenue this month"
        />
        <MetricCard 
          title="Users" 
          value={data.users.toLocaleString()} 
          description="Active users"
        />
        <MetricCard 
          title="Orders" 
          value={data.orders.toLocaleString()} 
          description="Total orders"
        />
      </div>
    </div>
  );
}

function MetricCard({ title, value, description }: { 
  title: string; 
  value: string; 
  description: string;
}) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h3 className="text-sm font-medium text-gray-500 mb-2">{title}</h3>
      <p className="text-3xl font-bold text-gray-900 mb-2">{value}</p>
      <p className="text-sm text-gray-600">{description}</p>
    </div>
  );
}
```

### On-Demand Revalidation

Revalidate data on demand based on specific events:

```tsx
// app/actions/revalidate.ts
'use server';

import { revalidatePath } from 'next/cache';

export async function revalidatePosts() {
  // Revalidate the posts page
  revalidatePath('/posts');
  revalidatePath('/blog');
  
  return { success: true };
}

export async function revalidateProduct(productId: string) {
  // Revalidate specific product page
  revalidatePath(`/products/${productId}`);
  revalidatePath('/products'); // Also revalidate product list
  
  return { success: true };
}
```

**Using revalidation in a form:**

```tsx
// components/PostEditor.tsx
"use client";

import { useState } from 'react';
import { revalidatePosts } from '@/app/actions/revalidate';

export function PostEditor() {
  const [title, setTitle] = useState('');
  const [content, setContent] = useState('');
  const [isSubmitting, setIsSubmitting] = useState(false);

  const handleSubmit = async (e: React.FormEvent) => {
    e.preventDefault();
    setIsSubmitting(true);

    try {
      // Create new post
      await fetch('/api/posts', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ title, content }),
      });

      // Revalidate posts page
      await revalidatePosts();

      // Clear form
      setTitle('');
      setContent('');
    } catch (error) {
      console.error('Error creating post:', error);
    } finally {
      setIsSubmitting(false);
    }
  };

  return (
    <form onSubmit={handleSubmit} className="space-y-6">
      <div>
        <label className="block text-sm font-medium text-gray-700 mb-2">
          Title
        </label>
        <input
          type="text"
          value={title}
          onChange={(e) => setTitle(e.target.value)}
          className="w-full px-4 py-2 border border-gray-300 rounded-lg"
          required
        />
      </div>

      <div>
        <label className="block text-sm font-medium text-gray-700 mb-2">
          Content
        </label>
        <textarea
          value={content}
          onChange={(e) => setContent(e.target.value)}
          rows={6}
          className="w-full px-4 py-2 border border-gray-300 rounded-lg"
          required
        />
      </div>

      <button
        type="submit"
        disabled={isSubmitting}
        className="w-full bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700 disabled:opacity-50"
      >
        {isSubmitting ? 'Creating...' : 'Create Post'}
      </button>
    </form>
  );
}
```

### Tag-Based Revalidation

Use tags for more granular cache control:

```tsx
// lib/api.ts

// Fetch posts with a specific tag
export async function getPosts() {
  const response = await fetch('https://api.example.com/posts', {
    next: { tags: ['posts'] }, // Tag this data
  });
  return response.json();
}

// Fetch a specific post with both individual and collection tags
export async function getPost(id: string) {
  const response = await fetch(`https://api.example.com/posts/${id}`, {
    next: { tags: [`post-${id}`, 'posts'] }, // Multiple tags
  });
  return response.json();
}

// Fetch related posts
export async function getRelatedPosts(categoryId: string) {
  const response = await fetch(
    `https://api.example.com/posts?category=${categoryId}`,
    {
      next: { tags: [`category-${categoryId}`, 'posts'] },
    }
  );
  return response.json();
}
```

**Revalidate by tag:**

```tsx
// app/actions/revalidate.ts
'use server';

import { revalidateTag } from 'next/cache';

export async function revalidatePost(postId: string) {
  // Revalidate specific post and all posts
  revalidateTag(`post-${postId}`);
  revalidateTag('posts');
}

export async function revalidateCategory(categoryId: string) {
  // Revalidate all posts in this category
  revalidateTag(`category-${categoryId}`);
}
```

### Cache Behavior Examples

**Different caching strategies for different data types:**

```tsx
// app/dashboard/page.tsx

// Static data that rarely changes (logo, company info)
async function getStaticData() {
  return await fetch('https://api.example.com/static', {
    cache: 'force-cache',
  }).then(res => res.json());
}

// Data that changes periodically (daily statistics)
async function getDailyStats() {
  return await fetch('https://api.example.com/stats/daily', {
    next: { revalidate: 86400 }, // 24 hours
  }).then(res => res.json());
}

// Real-time data (stock prices, live scores)
async function getRealTimeData() {
  return await fetch('https://api.example.com/realtime', {
    cache: 'no-store',
  }).then(res => res.json());
}

// User-specific data (never cache)
async function getUserData(userId: string) {
  return await fetch(`https://api.example.com/users/${userId}`, {
    cache: 'no-store',
  }).then(res => res.json());
}

export default async function DashboardPage() {
  const [staticData, dailyStats, realTimeData] = await Promise.all([
    getStaticData(),
    getDailyStats(),
    getRealTimeData(),
  ]);

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">
        Dashboard
      </h1>

      <div className="space-y-6">
        <div className="bg-white rounded-lg shadow-md p-6">
          <h2 className="text-xl font-semibold mb-4">Static Data</h2>
          <pre>{JSON.stringify(staticData, null, 2)}</pre>
        </div>

        <div className="bg-white rounded-lg shadow-md p-6">
          <h2 className="text-xl font-semibold mb-4">Daily Statistics</h2>
          <pre>{JSON.stringify(dailyStats, null, 2)}</pre>
        </div>

        <div className="bg-white rounded-lg shadow-md p-6">
          <h2 className="text-xl font-semibold mb-4">Real-Time Data</h2>
          <pre>{JSON.stringify(realTimeData, null, 2)}</pre>
        </div>
      </div>
    </div>
  );
}
```

## 8.4 Static vs Dynamic Data

Understanding the difference between static and dynamic data is crucial for choosing the right rendering and caching strategy.

### Static Data

Static data doesn't change frequently and can be cached for long periods:

```tsx
// app/about/page.tsx

// This data is static and can be cached indefinitely
async function getCompanyInfo() {
  const response = await fetch('https://api.example.com/company', {
    cache: 'force-cache',
  });
  return response.json();
}

export default async function AboutPage() {
  const companyInfo = await getCompanyInfo();

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="max-w-3xl mx-auto">
        <h1 className="text-3xl font-bold text-gray-900 mb-8">
          About {companyInfo.name}
        </h1>

        <div className="prose prose-lg">
          <p className="text-gray-600 mb-6">
            {companyInfo.description}
          </p>

          <div className="grid grid-cols-1 md:grid-cols-3 gap-6 mt-8">
            <StatCard 
              label="Founded" 
              value={companyInfo.founded} 
            />
            <StatCard 
              label="Employees" 
              value={companyInfo.employees} 
            />
            <StatCard 
              label="Locations" 
              value={companyInfo.locations} 
            />
          </div>
        </div>
      </div>
    </div>
  );
}

function StatCard({ label, value }: { label: string; value: string | number }) {
  return (
    <div className="bg-white rounded-lg shadow-md p-6 text-center">
      <p className="text-sm text-gray-500 mb-2">{label}</p>
      <p className="text-2xl font-bold text-gray-900">{value}</p>
    </div>
  );
}
```

### Dynamic Data

Dynamic data changes frequently and requires fresh data on each request or frequent revalidation:

```tsx
// app/dashboard/activity/page.tsx

// This data changes frequently and needs to be fresh
async function getRecentActivity() {
  const response = await fetch('https://api.example.com/activity/recent', {
    cache: 'no-store', // Always get fresh data
  });
  return response.json();
}

export default async function ActivityPage() {
  const activities = await getRecentActivity();

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">
        Recent Activity
      </h1>

      <div className="bg-white rounded-lg shadow-md overflow-hidden">
        <ul className="divide-y divide-gray-200">
          {activities.map((activity: any) => (
            <li key={activity.id} className="p-6">
              <div className="flex items-center space-x-4">
                <div className="flex-shrink-0">
                  <div className="w-10 h-10 bg-blue-100 rounded-full flex items-center justify-center">
                    <span className="text-blue-600 font-semibold">
                      {activity.user[0]}
                    </span>
                  </div>
                </div>
                <div className="flex-1">
                  <p className="text-sm font-medium text-gray-900">
                    {activity.user}
                  </p>
                  <p className="text-sm text-gray-600">
                    {activity.action}
                  </p>
                </div>
                <div className="text-sm text-gray-500">
                  {new Date(activity.timestamp).toLocaleString()}
                </div>
              </div>
            </li>
          ))}
        </ul>
      </div>
    </div>
  );
}
```

### Hybrid Approach

Mix static and dynamic data for optimal performance:

```tsx
// app/products/page.tsx

// Static data - categories and filters
async function getProductCategories() {
  return await fetch('https://api.example.com/categories', {
    cache: 'force-cache',
  }).then(res => res.json());
}

// Dynamic data - product listings with frequent updates
async function getProducts(filters?: string) {
  return await fetch(
    `https://api.example.com/products${filters ? `?${filters}` : ''}`,
    {
      next: { revalidate: 300 }, // Revalidate every 5 minutes
    }
  ).then(res => res.json());
}

export default async function ProductsPage({
  searchParams,
}: {
  searchParams?: { [key: string]: string | string[] | undefined };
}) {
  const [categories, products] = await Promise.all([
    getProductCategories(),
    getProducts(searchParams ? new URLSearchParams(searchParams as Record<string, string>).toString() : undefined),
  ]);

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Products</h1>

      <div className="flex flex-col lg:flex-row gap-8">
        {/* Sidebar - Static categories */}
        <aside className="w-full lg:w-64 flex-shrink-0">
          <div className="bg-white rounded-lg shadow-md p-6">
            <h2 className="text-lg font-semibold mb-4">Categories</h2>
            <ul className="space-y-2">
              {categories.map((category: any) => (
                <li key={category.id}>
                  <a
                    href={`/products?category=${category.id}`}
                    className="block px-3 py-2 rounded hover:bg-gray-100"
                  >
                    {category.name}
                  </a>
                </li>
              ))}
            </ul>
          </div>
        </aside>

        {/* Main content - Dynamic products */}
        <div className="flex-1">
          <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6">
            {products.map((product: any) => (
              <ProductCard key={product.id} product={product} />
            ))}
          </div>
        </div>
      </div>
    </div>
  );
}

function ProductCard({ product }: { product: any }) {
  return (
    <div className="bg-white rounded-lg shadow-md overflow-hidden hover:shadow-lg transition-shadow">
      <div className="aspect-video bg-gray-100">
        <img
          src={product.imageUrl}
          alt={product.name}
          className="w-full h-full object-cover"
        />
      </div>
      <div className="p-6">
        <h3 className="text-lg font-semibold text-gray-900 mb-2">
          {product.name}
        </h3>
        <p className="text-gray-600 mb-4">{product.description}</p>
        <div className="flex justify-between items-center">
          <span className="text-2xl font-bold text-blue-600">
            ${product.price.toFixed(2)}
          </span>
          <button className="px-4 py-2 bg-blue-600 text-white rounded hover:bg-blue-700">
            Add to Cart
          </button>
        </div>
      </div>
    </div>
  );
}
```

### Choosing the Right Strategy

**Use static data (force-cache) when:**

- Data rarely changes (company info, FAQ, terms of service)
- Performance is critical (marketing pages, landing pages)
- SEO is important (blog posts, documentation)

**Use time-based revalidation when:**

- Data changes on a predictable schedule (daily stats, weekly reports)
- Slight staleness is acceptable (product catalogs, user profiles)
- You want to balance freshness and performance

**Use no-store (dynamic) when:**

- Data changes in real-time (stock prices, live scores)
- User-specific data (personalized content, notifications)
- Security-sensitive data (authentication tokens, private data)

## 8.5 Error Handling and Loading States

Proper error handling and loading states are essential for providing a good user experience when fetching data.

### Loading States with Suspense

Use React's Suspense component for loading states:

```tsx
// app/dashboard/page.tsx

import { Suspense } from 'react';

// Slow component (simulates data fetching)
async function SlowComponent() {
  // Simulate slow data fetch
  await new Promise(resolve => setTimeout(resolve, 2000));
  
  const data = await fetch('https://api.example.com/data').then(res => res.json());
  
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h3 className="text-lg font-semibold mb-4">Slow Component</h3>
      <pre>{JSON.stringify(data, null, 2)}</pre>
    </div>
  );
}

// Fast component
async function FastComponent() {
  const data = await fetch('https://api.example.com/fast-data').then(res => res.json());
  
  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h3 className="text-lg font-semibold mb-4">Fast Component</h3>
      <pre>{JSON.stringify(data, null, 2)}</pre>
    </div>
  );
}

export default function DashboardPage() {
  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Dashboard</h1>

      <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
        {/* Fast component loads immediately */}
        <FastComponent />

        {/* Slow component shows loading state */}
        <Suspense fallback={<LoadingSkeleton />}>
          <SlowComponent />
        </Suspense>
      </div>
    </div>
  );
}

function LoadingSkeleton() {
  return (
    <div className="bg-white rounded-lg shadow-md p-6 animate-pulse">
      <div className="h-6 bg-gray-200 rounded w-1/3 mb-4"></div>
      <div className="space-y-2">
        <div className="h-4 bg-gray-200 rounded"></div>
        <div className="h-4 bg-gray-200 rounded w-5/6"></div>
        <div className="h-4 bg-gray-200 rounded w-4/6"></div>
      </div>
    </div>
  );
}
```

### Error Boundaries

Create error boundaries for graceful error handling:

```tsx
// app/dashboard/error.tsx
"use client";

interface ErrorProps {
  error: Error & { digest?: string };
  reset: () => void;
}

export default function DashboardError({ error, reset }: ErrorProps) {
  return (
    <div className="min-h-screen flex items-center justify-center bg-gray-50">
      <div className="max-w-md w-full bg-white rounded-lg shadow-lg p-8">
        <div className="text-center">
          <div className="mx-auto flex items-center justify-center h-12 w-12 rounded-full bg-red-100 mb-4">
            <svg className="h-6 w-6 text-red-600" fill="none" viewBox="0 0 24 24" stroke="currentColor">
              <path strokeLinecap="round" strokeLinejoin="round" strokeWidth={2} d="M12 9v2m0 4h.01m-6.938 4h13.856c1.54 0 2.502-1.667 1.732-3L13.732 4c-.77-1.333-2.694-1.333-3.464 0L3.34 16c-.77 1.333.192 3 1.732 3z" />
            </svg>
          </div>

          <h2 className="text-2xl font-bold text-gray-900 mb-4">
            Something went wrong!
          </h2>

          <p className="text-gray-600 mb-6">
            We encountered an error while loading the dashboard.
          </p>

          <div className="space-y-3">
            <button
              onClick={reset}
              className="w-full bg-blue-600 text-white px-6 py-3 rounded-lg hover:bg-blue-700 transition-colors"
            >
              Try Again
            </button>

            <a
              href="/"
              className="block w-full text-center text-blue-600 hover:text-blue-800 py-3"
            >
              Go to Home
            </a>
          </div>

          {process.env.NODE_ENV === 'development' && (
            <div className="mt-6 p-4 bg-gray-100 rounded-lg text-left">
              <p className="text-sm font-mono text-red-600 break-all">
                {error.message}
              </p>
            </div>
          )}
        </div>
      </div>
    </div>
  );
}
```

### Retry Logic

Implement retry logic for failed requests:

```tsx
// hooks/useFetchWithRetry.ts

import { useState, useEffect, useCallback } from 'react';

interface UseFetchWithRetryResult<T> {
  data: T | null;
  loading: boolean;
  error: string | null;
  retryCount: number;
  retry: () => void;
}

export function useFetchWithRetry<T>(
  url: string,
  maxRetries = 3,
  retryDelay = 1000
): UseFetchWithRetryResult<T> {
  const [data, setData] = useState<T | null>(null);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState<string | null>(null);
  const [retryCount, setRetryCount] = useState(0);

  const fetchData = useCallback(async () => {
    let currentRetry = 0;
    let lastError: string | null = null;

    while (currentRetry <= maxRetries) {
      try {
        setLoading(true);
        setError(null);

        const response = await fetch(url);

        if (!response.ok) {
          throw new Error(`HTTP error! status: ${response.status}`);
        }

        const result = await response.json();
        setData(result);
        setRetryCount(currentRetry);
        return;
      } catch (err) {
        lastError = err instanceof Error ? err.message : 'An error occurred';
        currentRetry++;

        if (currentRetry <= maxRetries) {
          // Wait before retrying
          await new Promise(resolve => setTimeout(resolve, retryDelay * currentRetry));
        }
      }
    }

    setError(lastError);
    setLoading(false);
  }, [url, maxRetries, retryDelay]);

  useEffect(() => {
    fetchData();
  }, [fetchData]);

  const retry = useCallback(() => {
    setRetryCount(0);
    fetchData();
  }, [fetchData]);

  return { data, loading, error, retryCount, retry };
}
```

**Using the retry hook:**

```tsx
// components/DataWithRetry.tsx
"use client";

import { useFetchWithRetry } from '@/hooks/useFetchWithRetry';

export function DataWithRetry() {
  const { data, loading, error, retryCount, retry } = useFetchWithRetry<any>(
    'https://api.example.com/data'
  );

  if (loading) {
    return (
      <div className="text-center py-12">
        <div className="inline-block animate-spin rounded-full h-12 w-12 border-b-2 border-blue-600"></div>
        {retryCount > 0 && (
          <p className="mt-4 text-sm text-gray-500">
            Retry attempt {retryCount + 1}...
          </p>
        )}
      </div>
    );
  }

  if (error) {
    return (
      <div className="bg-red-50 border border-red-200 rounded-lg p-6">
        <p className="text-red-700 mb-4">Error: {error}</p>
        <button
          onClick={retry}
          className="px-4 py-2 bg-red-600 text-white rounded hover:bg-red-700"
        >
          Retry ({retryCount} attempts)
        </button>
      </div>
    );
  }

  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <pre>{JSON.stringify(data, null, 2)}</pre>
    </div>
  );
}
```

## 8.6 Data Fetching Best Practices

Following best practices ensures your data fetching is efficient, reliable, and maintainable.

### 1. Minimize Data Fetching

Fetch only the data you need:

```tsx
// ❌ Bad - Fetching too much data
async function getAllUsers() {
  const response = await fetch('https://api.example.com/users');
  return response.json(); // Returns all fields including sensitive data
}

// ✅ Good - Fetching only needed fields
async function getUserList() {
  const response = await fetch('https://api.example.com/users?fields=id,name,email');
  return response.json(); // Only returns id, name, and email
}
```

### 2. Use Parallel Data Fetching

Fetch independent data in parallel:

```tsx
// ❌ Bad - Sequential fetching
async function getDashboardData() {
  const users = await fetchUsers();
  const posts = await fetchPosts();
  const comments = await fetchComments();
  return { users, posts, comments };
}

// ✅ Good - Parallel fetching
async function getDashboardData() {
  const [users, posts, comments] = await Promise.all([
    fetchUsers(),
    fetchPosts(),
    fetchComments(),
  ]);
  return { users, posts, comments };
}
```

### 3. Implement Proper Error Handling

Handle errors at every level:

```tsx
// ✅ Good - Comprehensive error handling
async function fetchWithHandling(url: string) {
  try {
    const response = await fetch(url);

    if (!response.ok) {
      // Handle different HTTP status codes
      if (response.status === 404) {
        throw new Error('Resource not found');
      } else if (response.status === 500) {
        throw new Error('Server error');
      } else {
        throw new Error(`HTTP error! status: ${response.status}`);
      }
    }

    return await response.json();
  } catch (error) {
    // Log error for debugging
    console.error('Error fetching data:', error);
    
    // Re-throw with user-friendly message
    throw new Error('Failed to load data. Please try again.');
  }
}
```

### 4. Cache Strategically

Choose appropriate caching strategies:

```tsx
// Static data - Long cache
async function getStaticContent() {
  return fetch('https://api.example.com/static', {
    cache: 'force-cache',
  }).then(res => res.json());
}

// Periodically updated data - Time-based revalidation
async function getDailyReports() {
  return fetch('https://api.example.com/reports', {
    next: { revalidate: 86400 }, // 24 hours
  }).then(res => res.json());
}

// Real-time data - No cache
async function getLivePrices() {
  return fetch('https://api.example.com/prices', {
    cache: 'no-store',
  }).then(res => res.json());
}
```

### 5. Optimize Bundle Size

Keep data fetching code efficient:

```tsx
// ✅ Good - Use dynamic imports for data fetching utilities
import dynamic from 'next/dynamic';

const DataVisualization = dynamic(
  () => import('@/components/DataVisualization'),
  { 
    loading: () => <p>Loading chart...</p>,
    ssr: false // Chart library only needed on client
  }
);

export default function AnalyticsPage() {
  return (
    <div>
      <h1>Analytics</h1>
      <DataVisualization />
    </div>
  );
}
```

### 6. Monitor Performance

Track data fetching performance:

```tsx
// lib/api.ts

export async function fetchWithMetrics<T>(url: string, options?: RequestInit): Promise<T> {
  const startTime = performance.now();
  
  try {
    const response = await fetch(url, options);
    const endTime = performance.now();
    
    // Log fetch duration
    console.log(`[API] ${url} - Duration: ${(endTime - startTime).toFixed(2)}ms`);
    
    if (!response.ok) {
      throw new Error(`HTTP error! status: ${response.status}`);
    }
    
    return await response.json();
  } catch (error) {
    const endTime = performance.now();
    console.error(`[API] ${url} - Failed after ${(endTime - startTime).toFixed(2)}ms`, error);
    throw error;
  }
}
```

## Key Takeaways from Chapter 8

1. **Server Component Data Fetching**: Server Components can fetch data directly on the server using async/await, providing excellent performance, SEO benefits, and eliminating client-side loading states.

2. **Client Component Data Fetching**: Client Components fetch data on the client using React hooks like `useEffect` and `useState`, which is useful for real-time data, user-specific content, and interactive features.

3. **Fetch Caching Strategies**: Next.js extends `fetch()` with caching options including `force-cache` (default), `no-store` (no caching), and time-based revalidation with `next: { revalidate }`.

4. **Static vs Dynamic Data**: Use static data (force-cache) for content that rarely changes, time-based revalidation for periodically updated data, and no-store for real-time or user-specific data.

5. **Error Handling and Loading States**: Implement proper error handling with try-catch blocks, use Suspense for loading states, create error boundaries for graceful error handling, and implement retry logic for failed requests.

6. **Data Fetching Best Practices**: Minimize data fetching by requesting only needed fields, use parallel fetching with `Promise.all()`, implement comprehensive error handling, cache strategically, optimize bundle size, and monitor performance.

## Coming Up Next

**Chapter 9: Optimizations Built-in**

Now that you understand data fetching, it's time to explore Next.js's powerful built-in optimization features. In Chapter 9, we'll learn about image optimization with next/image, font optimization with next/font, script optimization, automatic code splitting, bundle analysis, and performance monitoring. These features will help you build blazing-fast applications with minimal configuration.